<a href="https://colab.research.google.com/github/neerajthandayan/CourseProject/blob/main/Calculating_Bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Libraries

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
ss = pd.read_csv('https://raw.githubusercontent.com/neerajthandayan/CourseProject/main/stop_search.csv', index_col=0)
pop = pd.read_csv('https://raw.githubusercontent.com/neerajthandayan/CourseProject/main/pop_data.csv', index_col=0)

In [3]:
ss.head()

,Geography,SS_White,SS_Black,SS_Asian,SS_Other,SS_Mixed
0,Cleveland,28793.0,227.0,605.0,150.0,136.0
1,Durham,4848.0,15.0,15.0,10.0,14.0
2,Northumbria,42225.0,189.0,864.0,49.0,114.0
3,North East Region,75866.0,431.0,1484.0,209.0,264.0
4,Cheshire,5746.0,64.0,63.0,78.0,51.0


In [4]:
pop.head()

,Geography,White,Black,Asian,Other,Mixed
0,Avon and Somerset,1491970,30923,33005,15380,28277
1,Bedfordshire,476647,29725,81480,8753,18456
2,Cambridgeshire,726531,10174,39535,12572,16029
3,Cheshire,995998,3264,12794,5230,10423
4,Cleveland,526456,3156,17419,4434,5762


In [5]:
m1 = pop.merge(ss,on='Geography')
m1.head()

,Geography,White,Black,Asian,Other,Mixed,SS_White,SS_Black,SS_Asian,SS_Other,SS_Mixed
0,Avon and Somerset,1491970,30923,33005,15380,28277,13190.0,1010.0,252.0,34.0,454.0
1,Bedfordshire,476647,29725,81480,8753,18456,5853.0,1057.0,1483.0,56.0,407.0
2,Cambridgeshire,726531,10174,39535,12572,16029,11382.0,453.0,898.0,75.0,337.0
3,Cheshire,995998,3264,12794,5230,10423,5746.0,64.0,63.0,78.0,51.0
4,Cleveland,526456,3156,17419,4434,5762,28793.0,227.0,605.0,150.0,136.0


#### Calculating Disproportionality Index

In [8]:
tot = m1.iloc[:,1:6].sum(axis=1)

for i in m1.columns[1:6]:
  m1[i] = (m1[i].copy() / tot)*100 

m1.head()

,Geography,White,Black,Asian,Other,Mixed,SS_White,SS_Black,SS_Asian,SS_Other,SS_Mixed
0,Avon and Somerset,93.274067,1.933225,2.063386,0.961517,1.767804,13190.0,1010.0,252.0,34.0,454.0
1,Bedfordshire,77.495891,4.832854,13.247467,1.423111,3.000678,5853.0,1057.0,1483.0,56.0,407.0
2,Cambridgeshire,90.270128,1.264101,4.912150,1.562048,1.991573,11382.0,453.0,898.0,75.0,337.0
3,Cheshire,96.914399,0.317600,1.244905,0.508899,1.014198,5746.0,64.0,63.0,78.0,51.0
4,Cleveland,94.477834,0.566376,3.126015,0.795726,1.034049,28793.0,227.0,605.0,150.0,136.0


In [9]:
tot1 = m1.iloc[:,6:].sum(axis=1)

for i in m1.columns[6:]:
  m1[i] = (m1[i].copy() / tot1)*100 

m1.head()

,Geography,White,Black,Asian,Other,Mixed,SS_White,SS_Black,SS_Asian,SS_Other,SS_Mixed
0,Avon and Somerset,93.274067,1.933225,2.063386,0.961517,1.767804,88.286479,6.760375,1.686747,0.227577,3.038822
1,Bedfordshire,77.495891,4.832854,13.247467,1.423111,3.000678,66.090786,11.935411,16.745709,0.632340,4.595754
2,Cambridgeshire,90.270128,1.264101,4.912150,1.562048,1.991573,86.588056,3.446177,6.831495,0.570559,2.563712
3,Cheshire,96.914399,0.317600,1.244905,0.508899,1.014198,95.734755,1.066311,1.049650,1.299567,0.849717
4,Cleveland,94.477834,0.566376,3.126015,0.795726,1.034049,96.262245,0.758918,2.022667,0.501488,0.454682


In [37]:
disp = pd.DataFrame(m1.iloc[:,6:].values /  m1.iloc[:,1:6].values)
ardisp = pd.concat([m1['Geography'],disp], axis=1)
ardisp.columns = m1.columns[:6]
ardisp['Bias'] = ardisp.iloc[:,1:].var(axis=1)
ardisp.sort_values(by='Bias', ascending=False).head()

,Geography,White,Black,Asian,Other,Mixed,Bias
8,Dorset,0.965319,10.135002,0.806613,0.368937,1.139970,17.434857
41,Wiltshire,0.941624,6.049953,0.959591,1.739736,1.461315,4.674281
38,West Mercia,0.934221,5.622370,3.040460,0.203685,2.053606,4.470477
35,Sussex,0.955878,4.887564,1.456618,0.450236,1.045649,3.186521
12,Gloucestershire,0.933222,4.952699,1.479978,1.295966,2.355977,2.635513


In [39]:
me = []

for i in range(len(ardisp)):
  ethni = ardisp.columns[1:-1]
  row = ardisp.iloc[i,1:-1].values
  me.append(ethni[np.argmax(row)])

In [42]:
ardisp['Target_Group'] = me
ardisp.sort_values(by='Bias', ascending=False).head()

,Geography,White,Black,Asian,Other,Mixed,Bias,Target_Group
8,Dorset,0.965319,10.135002,0.806613,0.368937,1.139970,17.434857,Black
41,Wiltshire,0.941624,6.049953,0.959591,1.739736,1.461315,4.674281,Black
38,West Mercia,0.934221,5.622370,3.040460,0.203685,2.053606,4.470477,Black
35,Sussex,0.955878,4.887564,1.456618,0.450236,1.045649,3.186521,Black
12,Gloucestershire,0.933222,4.952699,1.479978,1.295966,2.355977,2.635513,Black
